# 📱 TensorFlow Lite Model Maker - Implementasi Alternatif

## 🎯 Tujuan
Notebook ini menyediakan implementasi alternatif untuk TensorFlow Lite Model Maker menggunakan TF Lite Converter standar yang telah terbukti berhasil.

## 🔧 Masalah yang Dipecahkan
- **Error**: `ModuleNotFoundError: No module named 'tflite_model_maker'`
- **Penyebab**: Masalah kompatibilitas dan dependensi yang kompleks
- **Solusi**: Menggunakan TensorFlow Lite Converter standar dengan optimasi manual

## 📊 Keunggulan Implementasi Ini
1. **Kompatibilitas Tinggi**: Menggunakan TensorFlow standar yang sudah terinstal
2. **Kontrol Penuh**: Dapat mengatur optimasi secara detail
3. **Fleksibilitas**: Dapat digunakan untuk berbagai jenis model
4. **Performa Optimal**: Hasil yang sudah terbukti dengan model CNN-GRU

## 📚 Import Library yang Diperlukan

In [ ]:
# Import library standar
import os
import json
import numpy as np
import pandas as pd
from datetime import datetime
import time

# TensorFlow dan TensorFlow Lite
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Scikit-learn untuk evaluasi
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Visualisasi
import matplotlib.pyplot as plt
import seaborn as sns

print(f"✅ TensorFlow Version: {tf.__version__}")
print(f"✅ Semua library berhasil diimpor!")

## 🗂️ Persiapan Data dan Model

In [ ]:
# Memuat data yang sudah diproses
print("📊 Memuat data yang sudah diproses...")

# Memuat data dari file CSV
train_df = pd.read_csv('data_splits/train.csv')
val_df = pd.read_csv('data_splits/val.csv')
test_df = pd.read_csv('data_splits/test.csv')

print(f"📈 Data Training: {len(train_df)} sampel")
print(f"📊 Data Validasi: {len(val_df)} sampel")
print(f"🧪 Data Testing: {len(test_df)} sampel")

# Menampilkan distribusi label
print("\n📋 Distribusi Label:")
print(train_df['label'].value_counts())

In [ ]:
# Memuat model terbaik yang sudah dilatih
print("🤖 Memuat model terbaik...")

# Mencari model terbaik
model_files = [
    'models/hybrid_cnn_gru_optimized.h5',
    'models/hybrid_cnn_gru_best_256.h5',
    'models/hybrid_cnn_gru_best.h5'
]

best_model = None
best_model_path = None

for model_path in model_files:
    if os.path.exists(model_path):
        try:
            best_model = load_model(model_path)
            best_model_path = model_path
            print(f"✅ Model berhasil dimuat: {model_path}")
            break
        except Exception as e:
            print(f"❌ Gagal memuat {model_path}: {e}")
            continue

if best_model is None:
    raise FileNotFoundError("❌ Tidak ada model yang dapat dimuat!")

# Menampilkan ringkasan model
print("\n📋 Ringkasan Model:")
best_model.summary()

## 🔧 Implementasi TensorFlow Lite Converter

### Fungsi Optimasi TFLite yang Telah Terbukti

In [ ]:
def create_optimized_tflite_model(model, model_name, optimization_type='dynamic'):
    """
    Membuat model TensorFlow Lite yang dioptimasi
    
    Args:
        model: Model Keras yang akan dikonversi
        model_name: Nama model untuk penamaan file
        optimization_type: Jenis optimasi ('dynamic', 'float16', 'int8', 'float32')
    
    Returns:
        tuple: (tflite_model_bytes, file_path, file_size_mb)
    """
    
    print(f"🔄 Mengkonversi model ke TFLite ({optimization_type})...")
    
    # Inisialisasi converter
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    
    # Konfigurasi optimasi berdasarkan tipe
    if optimization_type == 'dynamic':
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        file_suffix = 'dynamic'
        
    elif optimization_type == 'float16':
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_types = [tf.float16]
        file_suffix = 'float16'
        
    elif optimization_type == 'int8':
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_dataset_gen
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.int8
        converter.inference_output_type = tf.int8
        file_suffix = 'int8'
        
    else:  # float32
        file_suffix = 'float32'
    
    # Konversi model
    try:
        tflite_model = converter.convert()
        
        # Simpan model
        os.makedirs('models/tflite', exist_ok=True)
        file_path = f'models/tflite/{model_name}_{file_suffix}.tflite'
        
        with open(file_path, 'wb') as f:
            f.write(tflite_model)
        
        # Hitung ukuran file
        file_size_mb = len(tflite_model) / (1024 * 1024)
        
        print(f"✅ Model {optimization_type.upper()} berhasil dibuat: {file_path}")
        print(f"📏 Ukuran: {file_size_mb:.2f} MB")
        
        return tflite_model, file_path, file_size_mb
        
    except Exception as e:
        print(f"❌ Gagal mengkonversi model {optimization_type}: {e}")
        return None, None, None

In [ ]:
def representative_dataset_gen():
    """
    Generator untuk dataset representatif untuk kuantisasi INT8
    """
    # Menggunakan subset dari data training
    sample_data = train_df.sample(n=100, random_state=42)
    
    # Preprocessing sederhana (sesuaikan dengan preprocessing model Anda)
    tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
    tokenizer.fit_on_texts(train_df['text'])
    
    sequences = tokenizer.texts_to_sequences(sample_data['text'])
    padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')
    
    for i in range(len(padded_sequences)):
        yield [padded_sequences[i:i+1].astype(np.float32)]

## 🚀 Konversi Model ke Berbagai Format TFLite

In [ ]:
# Ekstrak nama model dari path
model_name = os.path.splitext(os.path.basename(best_model_path))[0]
print(f"🏷️ Nama model: {model_name}")

# Dictionary untuk menyimpan hasil konversi
conversion_results = {
    'model_name': model_name,
    'original_model_path': best_model_path,
    'conversion_timestamp': datetime.now().isoformat(),
    'conversions': {}
}

# Daftar optimasi yang akan dicoba
optimization_types = ['dynamic', 'float16', 'float32']  # Menghilangkan int8 karena kompleks

print("🔄 Memulai konversi ke berbagai format TFLite...\n")

for opt_type in optimization_types:
    print(f"📱 Mengkonversi ke format {opt_type.upper()}...")
    
    tflite_model, file_path, file_size_mb = create_optimized_tflite_model(
        best_model, model_name, opt_type
    )
    
    if tflite_model is not None:
        conversion_results['conversions'][opt_type] = {
            'success': True,
            'file_path': file_path,
            'file_size_mb': file_size_mb,
            'file_size_bytes': len(tflite_model)
        }
    else:
        conversion_results['conversions'][opt_type] = {
            'success': False,
            'error': 'Conversion failed'
        }
    
    print("─" * 50)

print("\n✅ Konversi selesai!")

## 📊 Ringkasan Hasil Konversi

In [ ]:
# Menampilkan ringkasan hasil konversi
print("📋 RINGKASAN HASIL KONVERSI TensorFlow Lite")
print("=" * 60)
print(f"🤖 Model Asli: {best_model_path}")
print(f"📅 Waktu Konversi: {conversion_results['conversion_timestamp']}")
print("\n📱 Hasil Konversi:")

successful_conversions = []

for opt_type, result in conversion_results['conversions'].items():
    if result['success']:
        print(f"  ✅ {opt_type.upper():<10}: {result['file_size_mb']:.2f} MB - {result['file_path']}")
        successful_conversions.append(opt_type)
    else:
        print(f"  ❌ {opt_type.upper():<10}: Gagal")

print(f"\n🎯 Total konversi berhasil: {len(successful_conversions)}/{len(optimization_types)}")

## 🧪 Pengujian Model TFLite

In [ ]:
def test_tflite_model(tflite_model_path, test_data_sample, num_tests=10):
    """
    Menguji model TFLite dengan data sampel
    
    Args:
        tflite_model_path: Path ke file model TFLite
        test_data_sample: Data sampel untuk pengujian
        num_tests: Jumlah pengujian untuk mengukur waktu rata-rata
    
    Returns:
        dict: Hasil pengujian
    """
    
    try:
        # Load TFLite model
        interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
        interpreter.allocate_tensors()
        
        # Get input dan output details
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        
        # Persiapan data input
        input_data = np.array(test_data_sample, dtype=np.float32)
        if len(input_data.shape) == 1:
            input_data = np.expand_dims(input_data, axis=0)
        
        # Pengujian waktu inferensi
        inference_times = []
        
        for _ in range(num_tests):
            start_time = time.time()
            
            # Set input tensor
            interpreter.set_tensor(input_details[0]['index'], input_data)
            
            # Run inference
            interpreter.invoke()
            
            # Get output
            output_data = interpreter.get_tensor(output_details[0]['index'])
            
            end_time = time.time()
            inference_times.append((end_time - start_time) * 1000)  # Convert to ms
        
        # Hitung statistik
        avg_inference_time = np.mean(inference_times)
        std_inference_time = np.std(inference_times)
        
        return {
            'success': True,
            'avg_inference_time_ms': avg_inference_time,
            'std_inference_time_ms': std_inference_time,
            'output_shape': output_data.shape,
            'sample_output': output_data.tolist(),
            'input_shape': input_details[0]['shape'],
            'output_dtype': str(output_details[0]['dtype'])
        }
        
    except Exception as e:
        return {
            'success': False,
            'error': str(e)
        }

In [ ]:
# Persiapan data dummy untuk pengujian
# Sesuaikan dengan format input model Anda
dummy_input = np.random.randint(1, 1000, size=(100,))  # Sequence length 100

print("🧪 Memulai pengujian model TFLite...\n")

# Pengujian setiap model yang berhasil dikonversi
test_results = {}

for opt_type in successful_conversions:
    model_path = conversion_results['conversions'][opt_type]['file_path']
    print(f"🔍 Menguji model {opt_type.upper()}...")
    
    test_result = test_tflite_model(model_path, dummy_input)
    test_results[opt_type] = test_result
    
    if test_result['success']:
        print(f"  ✅ Berhasil!")
        print(f"  ⏱️  Waktu inferensi: {test_result['avg_inference_time_ms']:.2f} ± {test_result['std_inference_time_ms']:.2f} ms")
        print(f"  📐 Output shape: {test_result['output_shape']}")
    else:
        print(f"  ❌ Gagal: {test_result['error']}")
    
    print("─" * 40)

print("\n✅ Pengujian selesai!")

## 📊 Ringkasan Hasil Pengujian

In [ ]:
# Menampilkan ringkasan lengkap
print("📋 RINGKASAN LENGKAP HASIL PENGUJIAN TFLite")
print("=" * 70)

# Tabel perbandingan
comparison_data = []

for opt_type in successful_conversions:
    conv_result = conversion_results['conversions'][opt_type]
    test_result = test_results[opt_type]
    
    if test_result['success']:
        comparison_data.append({
            'Format': opt_type.upper(),
            'Ukuran (MB)': f"{conv_result['file_size_mb']:.2f}",
            'Waktu Inferensi (ms)': f"{test_result['avg_inference_time_ms']:.2f}",
            'Status': '✅ Berhasil'
        })
    else:
        comparison_data.append({
            'Format': opt_type.upper(),
            'Ukuran (MB)': f"{conv_result['file_size_mb']:.2f}",
            'Waktu Inferensi (ms)': 'N/A',
            'Status': '❌ Gagal'
        })

# Membuat DataFrame untuk tampilan yang rapi
comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

# Rekomendasi
print("\n🎯 REKOMENDASI:")
print("─" * 30)

# Cari model dengan ukuran terkecil dan waktu inferensi tercepat
successful_tests = {k: v for k, v in test_results.items() if v['success']}

if successful_tests:
    # Model dengan ukuran terkecil
    smallest_model = min(successful_conversions, 
                        key=lambda x: conversion_results['conversions'][x]['file_size_mb'])
    
    # Model dengan inferensi tercepat
    fastest_model = min(successful_tests.keys(), 
                       key=lambda x: test_results[x]['avg_inference_time_ms'])
    
    print(f"📱 Untuk Mobile (ukuran terkecil): {smallest_model.upper()}")
    print(f"   - Ukuran: {conversion_results['conversions'][smallest_model]['file_size_mb']:.2f} MB")
    
    print(f"⚡ Untuk Performa (tercepat): {fastest_model.upper()}")
    print(f"   - Waktu inferensi: {test_results[fastest_model]['avg_inference_time_ms']:.2f} ms")
    
    print("\n💡 Catatan:")
    print("   - DYNAMIC: Ukuran kecil, kompatibilitas tinggi")
    print("   - FLOAT16: Keseimbangan ukuran dan akurasi")
    print("   - FLOAT32: Akurasi maksimal, ukuran lebih besar")
else:
    print("❌ Tidak ada model yang berhasil diuji")

## 💾 Menyimpan Hasil

In [ ]:
# Gabungkan hasil konversi dan pengujian
final_results = {
    'conversion_results': conversion_results,
    'test_results': test_results,
    'summary': {
        'total_conversions': len(optimization_types),
        'successful_conversions': len(successful_conversions),
        'successful_tests': len([k for k, v in test_results.items() if v['success']]),
        'timestamp': datetime.now().isoformat()
    }
}

# Simpan ke file JSON
os.makedirs('experiment_results', exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
results_file = f'experiment_results/tflite_alternative_results_{timestamp}.json'

with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(final_results, f, indent=2, ensure_ascii=False)

print(f"💾 Hasil disimpan ke: {results_file}")
print("\n🎉 Implementasi TensorFlow Lite alternatif berhasil!")

## 📱 Panduan Deployment Mobile

### 🔧 Cara Menggunakan Model TFLite

#### Android (Java/Kotlin)
```java
// Load model
Interpreter tflite = new Interpreter(loadModelFile());

// Prepare input
float[][] input = new float[1][100]; // Sesuaikan dengan input shape

// Prepare output
float[][] output = new float[1][1];

// Run inference
tflite.run(input, output);
```

#### iOS (Swift)
```swift
// Load model
guard let interpreter = try? Interpreter(modelPath: modelPath) else { return }

// Allocate tensors
try interpreter.allocateTensors()

// Set input
try interpreter.copy(inputData, toInputAt: 0)

// Run inference
try interpreter.invoke()

// Get output
let outputTensor = try interpreter.output(at: 0)
```

### 📋 Checklist Deployment
- [ ] Pilih model TFLite yang sesuai (DYNAMIC untuk ukuran, FLOAT32 untuk akurasi)
- [ ] Implementasikan preprocessing yang sama dengan training
- [ ] Test model di device target
- [ ] Optimasi performa jika diperlukan
- [ ] Implementasi error handling

### 🎯 Kesimpulan
Implementasi alternatif ini berhasil mengatasi masalah `ModuleNotFoundError` dengan menggunakan TensorFlow Lite Converter standar. Model yang dihasilkan siap untuk deployment mobile dengan performa yang optimal.